# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

# Overview of the Agents Framework

### ## The Agents: Your Specialist Team

This crew is composed of four agents, each with a distinct role in the job application process.

* **`researcher` (Tech Job Researcher):** This agent's only job is to analyze the provided job posting. It uses its search and scrape tools to extract all the key requirements, such as necessary skills, qualifications, and years of experience, that the employer is looking for.
* **`profiler` (Personal Profiler for Engineers):** This agent focuses on the job applicant. It takes the applicant's information, like a GitHub profile and a personal write-up, and uses its tools to create a comprehensive professional profile. This includes a list of skills, project experiences, and contributions.
* **`resume_strategist` (Resume Strategist for Engineers):** This is the resume writer. Its role is to take the output from the first two agents (the job requirements and the applicant's profile) and rewrite the resume. Its goal is to highlight the applicant's most relevant skills and experiences to perfectly match what the job posting is asking for.
* **`interview_preparer` (Engineering Interview Preparer):** This is the final agent in the chain. It takes the tailored resume and the original job requirements to create a list of potential interview questions and key talking points. This prepares the candidate to confidently discuss how their skills and experience align with the job.

### ## The Tasks: The Step-by-Step Workflow

Each agent is assigned a specific task that defines its work for a given run of the crew.

* **`research_task`:** Assigned to the `researcher`, this task's objective is to analyze the job posting URL and produce a structured list of all the job requirements. This task runs asynchronously.
* **`profile_task`:** Assigned to the `profiler`, this task's goal is to analyze the applicant's provided GitHub URL and personal write-up to create a detailed professional profile. This task also runs asynchronously.
* **`resume_strategy_task`:** Assigned to the `resume_strategist`, this task is dependent on the completion of the first two tasks, as specified by `context=[research_task, profile_task]`. It instructs the agent to rewrite the applicant's resume to align with the job requirements and saves the output to a file named `tailored_resume.md`.
* **`interview_preparation_task`:** Assigned to the `interview_preparer`, this is the final task. It depends on the completion of the previous three tasks (`context=[research_task, profile_task, resume_strategy_task]`). It instructs the agent to create interview questions and talking points based on all the information gathered and saves the result to a file named `interview_materials.md`.

# Information About the Tools


### ## `FileReadTool` 📄

**Purpose:** To read the entire content of a specific local file.

This tool is used when an agent needs to access information that is stored in a file on your machine, such as a resume, a set of instructions, or a pre-written template.

***

### ## `ScrapeWebsiteTool` 🕸️

**Purpose:** To extract all the text content from a given webpage URL.

This is useful when an agent needs to analyze the information on a specific webpage, like a job posting or a company's "About Us" page, without needing to perform a general web search.

***

### ## `MDXSearchTool` 🔍

**Purpose:** To perform a semantic search *within* a Markdown (`.md` or `.mdx`) file.

Instead of reading the whole file, this tool allows an agent to find the most relevant sections of a document based on the meaning of a query. For example, it can search a resume file for "experience with Python" and find the most relevant job descriptions, even if they don't use those exact words.

***

### ## `SerperDevTool` 🌐

**Purpose:** To perform a real-time Google search and get a list of results.

This is a general web search tool. Agents use it to find current information, news, or articles about a company, person, or topic when they don't have a specific URL to scrape.

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [5]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [6]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [7]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.10it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [13]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


## Creating Agents

In [14]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [15]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [16]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [17]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [18]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [19]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [20]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [21]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [22]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [23]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [24]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

Action Input: {"search_query": "AI Fund job posting 6c82e23e-d954-4dd8-a734-c0c2c5ee00f1"} 


Search results: Title: Careers - AI Fund
Link: https://aifund.ai/careers/
Snippet: Join our ecosystem of seasoned business leaders, innovators, and engineers, and help us build the future. Let's build the future together.
---
Title: Jobs at AI Fund Companies | AI Fund
Link: https://careers.aifund.ai/jobs/ai-fund
Snippet: Posted Anytime. Powered by Consider. 12 jobs. Technical Founder in Residence. Marketing StrategyInfrastructureApplication ArchitectureMachine LearningCompliance ...
---
Title: AI Fund - career jobs - Lever
Link: https://jobs.lever.co/AIFund
Snippet: Head of Marketing. Hybrid — Full timeSan Francisco, California ; Product Marketing Manager. Remote — Full timeSan Francisco Bay Area, Remote ; UI/UX Designer.
---
Title: crewAI-quickstart/crewai_example_job_application_quickstart.ipynb ...
Link: https://github.com/alexfazio/crewAI-quickstart/blob/main/crewai_example_job_application

Final Answer: 

**Noah's Profile**

Noah is an accomplished Software Engineering Leader with 18 years of experience, specializing in managing remote and in-office teams. He is an expert in multiple programming languages and frameworks. Noah holds an MBA and has a strong background in AI and data science. He has successfully led major tech initiatives and startups, demonstrating his ability to drive innovation and growth in the tech industry. Noah is ideal for leadership roles that require a strategic and innovative approach.

**Skills:**
- Software Engineering Leadership
- Team Management (remote and in-office)
- Expertise in multiple programming languages and frameworks
- AI and Data Science
- Strategic Planning
- Innovation and Growth

**Project Experiences:**
- Led major tech initiatives and startups
- Orchestrated role-playing, autonomous AI agents with CrewAI
- Developed a gamification gem for Ruby on Rails applications
- Implemented a ranking algorithm to Ruby on Rails applicatio

I need to gather relevant information from Noah's profile and work experiences to create tailored interview questions and talking points based on his skills and experiences.

Action: Read a file's content
Action Input: {} 


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of

- Dislplay the generated `tailored_resume.md` file.

In [25]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is an accomplished Software Engineering Leader with 18 years of experience, specializing in managing remote and in-office teams. He holds an MBA and has a strong background in AI and data science. Noah is ideal for leadership roles that require a strategic and innovative approach.

## Skills
- Software Engineering Leadership
- Team Management (remote and in-office)
- Expertise in multiple programming languages and frameworks
- AI and Data Science
- Strategic Planning
- Innovation and Growth

## Work Experience
### Director of Software Engineering - DataKernel (2022 - Present)
- Transformed the engineering division into a key revenue pillar for DataKernel.
- Spearheaded the integration of cutting-edge AI technologies and scalable vector databases.
- Focused on achieving strategic project goals that influenced the company's direction.

### Senior Software Engineering Manager - DataKernel (2019 - 2022)
- Directed engineering strategy in collaboration with C-level executives.
- Managed diverse teams across multiple time zones, enhancing team performance.

### Founder & CEO - InnovPet (2019 - 2022)
- Founded a startup focused on innovative IoT solutions for pet care.
- Oversaw product development and successful market entry.

### Engineering Manager - EliteDevs (2018 - 2019)
- Formulated and executed strategic plans to enhance inter-departmental coordination.
- Managed multiple engineering teams to balance productivity with innovation.

### Engineering Manager - PrintPack (2016 - 2018)
- Led the development of a high-performance engineering team, increasing company revenue by 500%.
- Integrated data analytics into business decision-making processes.

## Education
- MBA in Information Technology from London Business School
- Advanced Leadership Techniques Certification from University of London
- Data Science Specialization Certification from Coursera (Johns Hopkins University)
- B.Sc. in Computer Science from University of Edinburgh

Noah Williams is well-equipped to excel in senior executive roles, bringing a unique blend of technical expertise and strategic leadership to any organization.

- Dislplay the generated `interview_materials.md` file.

In [26]:
display(Markdown("./interview_materials.md"))

Based on Noah Williams' profile, work experiences, skills, and contributions, here are some tailored interview questions and talking points for potential leadership roles in the tech industry:

1. Software Engineering Leadership:
- Can you share a specific example of how you have successfully led engineering teams, both remote and in-office, to achieve key project milestones?
- How do you approach team management to ensure high performance and collaboration across different time zones?

2. Expertise in Multiple Programming Languages and Frameworks:
- What programming languages and frameworks do you consider your strongest suit, and how have you leveraged them in your previous roles?
- Can you provide an example of a project where your diverse programming expertise was crucial to its success?

3. AI and Data Science:
- How have you applied your background in AI and data science to drive innovation and growth within the tech industry?
- Can you discuss a specific AI project you led that demonstrates your ability to implement cutting-edge technologies effectively?

4. Strategic Planning and Innovation:
- What role have strategic planning and innovation played in shaping your leadership style and approach to tech initiatives?
- How do you ensure that your strategic decisions align with the company's long-term goals and vision?

5. Project Experiences and Contributions:
- Could you elaborate on a major tech initiative or startup you led, highlighting the challenges faced and the outcomes achieved?
- As the founder of crewAI, what lessons did you learn from building and growing this startup that you would apply to future ventures?

6. Communication Style and Leadership:
- How do you approach communication with your team, ensuring that your directives are clear and goals are well-defined?
- In what ways do you empower and collaborate with your team members to foster a culture of innovation and growth?

7. Interests and Passion for Technology:
- What specific aspects of technology and entrepreneurship drive your passion, and how do they influence your professional decisions?
- Can you discuss a tough, interesting problem you encountered in your career and the high-impact solution you implemented?

These questions and talking points aim to delve deeper into Noah Williams' experiences, skills, and leadership abilities, allowing him to showcase his strengths and alignment with the requirements of leadership roles in the tech industry.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)